In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

GPU Available: False


In [ ]:
# Install Ultralytics (YOLOv11)
!pip install ultralytics -q

import ultralytics
ultralytics.checks()

In [ ]:
# Set paths - MODIFY THIS to match your Google Drive path
import os
dataset_path = '/content/drive/MyDrive/military-aircraft-yolo'

# Verify dataset exists
assert os.path.exists(dataset_path), f"Dataset not found at {dataset_path}. Upload your dataset folder to Google Drive first!"
print(f" Dataset found at: {dataset_path}")
print(f" Labels folder: {os.path.exists(os.path.join(dataset_path, 'labels'))}")
print(f" Images folder: {os.path.exists(os.path.join(dataset_path, 'images'))}")
print(f" Config file: {os.path.exists(os.path.join(dataset_path, 'data.yaml'))}")

In [ ]:
# View dataset configuration
import yaml

with open(f'{dataset_path}/data.yaml', 'r') as f:
    config = yaml.safe_load(f)
    
print("Dataset Configuration:")
print(f"Number of classes: {config['nc']}")
print(f"Class names: {config['names']}")

In [ ]:
# Update data.yaml path to point to current location
config['path'] = dataset_path
config['train'] = 'images'  # Images folder for training
config['val'] = 'images'    # Same folder for validation (YOLO will auto-split)

# Save updated config
with open(f'{dataset_path}/data.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)
    
print(" Updated data.yaml with correct paths")

In [ ]:
# Setup training directory in Google Drive for persistent checkpoints
import os
from pathlib import Path

# Use Google Drive for training output so checkpoints persist across runtime restarts
training_dir = '/content/drive/MyDrive/military-aircraft-training'
os.makedirs(training_dir, exist_ok=True)

# Check for existing checkpoint
checkpoint_path = Path(f'{training_dir}/weights/last.pt')
resume_training = False

print("\n" + "=" * 60)
print("CHECKPOINT STATUS")
print("=" * 60)

if checkpoint_path.exists():
    file_size = checkpoint_path.stat().st_size / (1024 * 1024)
    print(f"✓ Found checkpoint: {checkpoint_path}")
    print(f"  Size: {file_size:.2f} MB")
    resume_choice = input("\nResume from checkpoint? (yes/no): ").lower()
    resume_training = resume_choice == 'yes'
    
    if resume_training:
        print("\n✓ Will RESUME training from last checkpoint")
    else:
        print("\n✓ Will START FRESH (existing files will be overwritten)")
else:
    print(f"ℹ No checkpoint found")
    print(f"  Training will save to: {training_dir}")
    print("  → Checkpoints will persist in Google Drive even if runtime restarts")
    
print("=" * 60 + "\n")

In [ ]:
# Train YOLOv11n model with Google Drive checkpointing
from ultralytics import YOLO

# Load model - either fresh or from checkpoint
if resume_training and checkpoint_path.exists():
    print(f"Loading checkpoint from {checkpoint_path}")
    model = YOLO(str(checkpoint_path))  # Load from checkpoint to resume
else:
    print("Loading pretrained YOLOv11n model")
    model = YOLO('yolo11n.pt')  # Load fresh pretrained model

# Train the model - saves directly to Google Drive
results = model.train(
    data=f'{dataset_path}/data.yaml',
    epochs=100,                # Number of training epochs
    imgsz=640,                 # Image size
    batch=32,                  # Batch size (32 for H100/A100, reduce to 16 for V100/L4)
    name='',                   # No subfolder name (save directly to project dir)
    patience=20,               # Early stopping patience
    save=True,                 # Save checkpoints
    save_period=5,             # Save checkpoint every 5 epochs to Google Drive
    plots=True,                # Generate plots
    device=0,                  # Use GPU 0
    workers=8,                 # Dataloader workers
    project=training_dir,      # Save to Google Drive directory
    exist_ok=True,             # Overwrite existing project
    pretrained=True if not resume_training else False,  # Use pretrained weights only for fresh training
    resume=resume_training,    # Resume from checkpoint if True
    optimizer='auto',         
    verbose=True,              # Verbose output
    seed=42,                   # Random seed for reproducibility
    lr0=0.01,                  # Initial learning rate
    lrf=0.01,                  # Final learning rate
    momentum=0.937,            # SGD momentum/Adam beta1
    weight_decay=0.0005,       # Weight decay
    warmup_epochs=3,           # Warmup epochs
    val=True,                  # Validate during training
    fraction=1.0,              # Use 100% of data
    cache=False                # Don't cache images (saves RAM)
)

print("\n✓ Training complete!")
print(f"✓ Models saved to Google Drive: {training_dir}")
print(f"✓ Best model: {training_dir}/weights/best.pt")
print(f"✓ Last checkpoint: {training_dir}/weights/last.pt")

In [ ]:
# View training results
from IPython.display import Image, display

print("Training Results:")
display(Image(filename=f'{training_dir}/results.png'))
display(Image(filename=f'{training_dir}/confusion_matrix.png'))

In [ ]:
# Validate the model
metrics = model.val()

print(f"\nmAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

In [ ]:
# Export the model (optional - for deployment)
model.export(format='onnx')  # Export to ONNX format
print("✓ Model exported to ONNX format")